<a href="https://colab.research.google.com/github/somayeh1404/cod/blob/main/so_com.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os
import numpy as np
import zipfile
from google.colab import files
import shutil


BATCH_SIZE = 32
NUM_FILTERS = 256
STEP_SIZE = 20
DROPOUT = 0.2
LEARNING_RATE = 0.0002
USE_BATCH_NORM = True
NUM_EPOCHS = 10


def setup_environment():

    !pip install -q kaggle


    uploaded = files.upload()


    !mkdir -p ~/.kaggle
    !mv kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json


    !kaggle datasets download -d mohamedhanyyy/chest-ctscan-images


    with zipfile.ZipFile('chest-ctscan-images.zip', 'r') as zip_ref:
        zip_ref.extractall('dataset')


    print("\nساختار دیتاست:")
    !ls -R dataset | head -20


def prepare_data():

    base_path = 'dataset/Data'
    train_path = os.path.join(base_path, 'train')
    test_path = os.path.join(base_path, 'test')


    train_transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.1),
        transforms.RandomRotation(15),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.RandomPerspective(distortion_scale=0.2, p=0.1),
        transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 0.5)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    test_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])


    train_dataset = datasets.ImageFolder(root=train_path, transform=train_transform)
    test_dataset = datasets.ImageFolder(root=test_path, transform=test_transform)


    print("\nاطلاعات دیتاست آموزشی:")
    print(f"تعداد کل نمونه‌ها: {len(train_dataset)}")
    print(f"تعداد کلاس‌ها: {len(train_dataset.classes)}")
    print(f"نام کلاس‌ها: {train_dataset.classes}")
    print(f"تعداد نمونه‌های هر کلاس: {np.bincount(train_dataset.targets)}")

    print("\nاطلاعات دیتاست تست:")
    print(f"تعداد کل نمونه‌ها: {len(test_dataset)}")
    print(f"تعداد کلاس‌ها: {len(test_dataset.classes)}")
    print(f"نام کلاس‌ها: {test_dataset.classes}")
    print(f"تعداد نمونه‌های هر کلاس: {np.bincount(test_dataset.targets)}")


    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

    return train_loader, test_loader, len(train_dataset.classes)


class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, NUM_FILTERS//4, kernel_size=3, padding=1),
            nn.BatchNorm2d(NUM_FILTERS//4) if USE_BATCH_NORM else nn.Identity(),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(NUM_FILTERS//4, NUM_FILTERS//2, kernel_size=3, padding=1),
            nn.BatchNorm2d(NUM_FILTERS//2) if USE_BATCH_NORM else nn.Identity(),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(NUM_FILTERS//2, NUM_FILTERS, kernel_size=3, padding=1),
            nn.BatchNorm2d(NUM_FILTERS) if USE_BATCH_NORM else nn.Identity(),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((1, 1))
        )

        self.classifier = nn.Sequential(
            nn.Dropout(DROPOUT),
            nn.Linear(NUM_FILTERS, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


def train_model(model, train_loader, test_loader, num_classes):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=0.1)

    best_accuracy = 0.0

    print("\nشروع آموزش مدل:")
    for epoch in range(NUM_EPOCHS):
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

        train_loss = train_loss / len(train_loader.dataset)
        train_accuracy = 100 * train_correct / train_total

        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * images.size(0)
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_loss = val_loss / len(test_loader.dataset)
        val_accuracy = 100 * val_correct / val_total

        scheduler.step()

        print(f"\nEpoch {epoch+1}/{NUM_EPOCHS}:")
        print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_accuracy:.2f}%")
        print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_accuracy:.2f}%")

        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            torch.save(model.state_dict(), 'best_model.pth')
            print(f"مدل بهتر ذخیره شد با دقت: {best_accuracy:.2f}%")

    return best_accuracy


def main():

    setup_environment()


    train_loader, test_loader, num_classes = prepare_data()


    model = CNNModel(num_classes=num_classes)


    best_acc = train_model(model, train_loader, test_loader, num_classes)


    print(f"\nبهترین دقت مدل روی داده تست: {best_acc:.2f}%")

if __name__ == "__main__":
    main()